In [ ]:
'''
L'objectif de ce projet est d'observer s'il existe des 'déserts sportifs', 
lieux en France où les infrastrcutures sportives manquent. 
La question sera alors d'essayer d'expliquer ces déserts sportifs, que ce soit par des raisons économiques ou politiques. 
Enfin, il s'agira de comparer la carte des déserts sportifs avec d'autres cartes connues, à l'instar des déserts médicaux.
'''

In [8]:
''' 
Il convient d'abord d'importer tous les modules python nécessaires au travail
'''

import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download
import requests
import zipfile
import io

In [99]:
'''
On importe ensuite nos jeux de données
Le jeu de données principal
'''
url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)

'''
Un jeu de données csv sur des informations économiques et démographiques au niveau des communes
'''

urlpop = "https://www.insee.fr/fr/statistiques/fichier/2521169/base_cc_comparateur_csv.zip"
response = requests.get(urlpop)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("base_cc_comparateur.csv") as csvfile:
        df_communes = pd.read_csv(csvfile, sep=";", low_memory=False)

'''
Jeu de données politiques au niveau des communes: résultats des législatives 2024 (2nd tour)
'''

url2="https://www.data.gouv.fr/api/1/datasets/r/5a8088fd-8168-402a-9f40-c48daab88cd1"
legislatives2=pd.read_csv(url2, sep=";", low_memory=False)


In [100]:
'''
Il s'agit maintenant de laver nos jeux de données afin de les réunir ensuite à l'aide des codes INSEE.
On conserve seulement les variables qui peuvent nous intéresser.
'''

cols = ["Nom de l'installation sportive", "Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Département Nom", "Densite Catégorie", "Nom de l'équipement sportif", "Type d'équipement sportif", "Longitude", "Latitude"]
equipement = equipement[cols]

cols=["CODGEO", "P22_POP", "NAISD24", "DECESD24", "P22_MEN","MED21", "TP6021", "P22_CHOM1564"]
df_communes = df_communes[cols]

cols=["Code commune", "Libellé commune"]
cols= cols + [(f"% Voix/exprimés {i}") for i in range(1,4)]+[ (f"Nuance candidat {i}") for i in range(1, 4)]
legislatives2= legislatives2[cols]

In [107]:
'''
On renomme maintenant les colonnes afin de pouvoir concatener les dataframes. 
'''

df_communes = df_communes.rename(columns={"CODGEO": "Commune INSEE"})
legislatives2 = legislatives2.rename(columns={"Code commune": "Commune INSEE"})

df_final = (
    equipement
    .merge(df_communes, on="Commune INSEE", how="left")
    .merge(legislatives2, on="Commune INSEE", how="left")
)

In [110]:
'''
On ajoute dans ce dataframe uniquement le parti politique en tête lors des élections de 2024, c'est-à-dire la nuance qui a eu
le plus de % Voix/exprimés. Pour calculer le max il faut donc remplacer les valeurs prises qui ne sont pour l'instant pas dans le bon format.
'''
cols_voix = ["% Voix/exprimés 1", "% Voix/exprimés 2", "% Voix/exprimés 3"]

for c in cols_voix:
    df_final[c] = (df_final[c].astype(str).str.replace("%", "", regex=False).str.replace(",", ".", regex=False).str.strip())

df_final["% Voix/exprimés 3"]=df_final["% Voix/exprimés 3"].str.replace("nan","0")

cols_valeurs = df_final.columns[19:22]
cols_associees = df_final.columns[22:25]

valeurs = df_final[cols_valeurs].to_numpy()
associees = df_final[cols_associees].to_numpy()
idx_max = np.argmax(valeurs, axis=1)
df_final["Score vainqueur"] = valeurs[np.arange(len(df_final)), idx_max]
df_final["Parti vainqueur"] = associees[np.arange(len(df_final)), idx_max]

In [128]:
'''
En faisant le test avec ma commune d'origine, on se rend compte que des lignes sont parfois en double, voire triple, on va donc supprimer ces doublons.
'''

test=df_final[df_final["Commune nom"] == "Eschau"]

df_final = df_final.drop_duplicates()

In [133]:
'''
Maintenant essayons de faire une carte de la france avec toutes les infrastructures sportives.
'''
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download

# --- 1️⃣ Nettoyer les données : garder uniquement les lignes avec coordonnées
df_final = df_final.dropna(subset=["Longitude", "Latitude"])

# --- 2️⃣ Créer le GeoDataFrame des points
gdf_pts = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final["Longitude"], df_final["Latitude"]),
    crs="EPSG:4326"  # Coordonnées géographiques WGS84
)

# --- 3️⃣ Télécharger un fond de carte (France entière)
# ✅ Ici : format GPKG (GeoPackage), plus stable que TopoJSON
shp_communes = carti_download(
    "France",                       # pas de 'values=' !
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="gpkg",       # ou "geojson" si tu préfères
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# --- 4️⃣ Charger le fond de carte dans GeoPandas
gdf_france = gpd.read_file(shp_communes)

# --- 5️⃣ Harmoniser les projections (Lambert-93 pour la France)
gdf_france = gdf_france.to_crs("EPSG:2154")
gdf_pts = gdf_pts.to_crs("EPSG:2154")

# --- 6️⃣ Afficher la carte
fig, ax = plt.subplots(figsize=(10, 12))

# Tracer le fond (contours des départements)
gdf_france.boundary.plot(ax=ax, linewidth=0.5, color="gray")

# Ajouter les points des infrastructures
gdf_pts.plot(ax=ax, markersize=3, alpha=0.6, color="red")

# Mise en forme
ax.set_title("Localisation des infrastructures sportives en France", fontsize=14)
ax.set_axis_off()
plt.show()



There was an error while reading the file from the URL: https://minio.lab.sspcloud.fr/projet-cartiflette/production/provider=IGN/dataset_family=ADMINEXPRESS/source=EXPRESS-COG-CARTO-TERRITOIRE/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE_DROM_RAPPROCHES=France/vectorfile_format=GPKG/territory=metropole/simplification=50/raw.GPKG
Error message: '/vsimem/pyogrio_b16dc7e26e0047288627a699859ea963' not recognized as being in a supported file format.; It might help to specify the correct driver explicitly by prefixing the file path with '<DRIVER>:', e.g. 'CSV:path'.


ValueError: All objects passed were None